In [3]:
import os
import pandas as pd
import re
import time
from google import genai
from dotenv import load_dotenv

load_dotenv(override=True)
GEMINI = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI)

In [4]:
df = pd.read_csv('data_filtered.csv')
print(f"Total komentar: {len(df)}")
print("\nContoh data awal:")
df.head()

Total komentar: 778

Contoh data awal:


,Video_ID,Teks_Komentar
0,MIo4tGN11j0,"Sempat mikir mau pindah ke negara sebelah, nge..."
1,MIo4tGN11j0,"Kalo kabur mau kemana ke Singapur ,emang di Si..."
2,MIo4tGN11j0,Klo sudah gelap susah terangnya lebih baik bubar
3,MIo4tGN11j0,Pengen kabur tapi gak punya uang😭
4,MIo4tGN11j0,Kalau mau pindah ke luar negeri ya silahkan sa...


In [5]:
def classify_sentiment_batch(comments):
    # Tambahkan penomoran pada setiap komentar
    numbered_comments = "\n".join([f"{i+1}. {c}" for i, c in enumerate(comments)])

    prompt = f"""
    Anda adalah seorang analis sentimen sosial Indonesia yang sangat teliti.
    Tugas Anda adalah menentukan sentimen dari setiap komentar berikut terhadap FENOMENA "kabur aja dulu" (pindah dari Indonesia).

    Tolong baca dengan cermat setiap komentar dan berikan penilaian yang akurat sesuai konteksnya.
    Jumlah komentar ada {len(comments)}.

    Kriteria Penilaian Sentimen:
    - Positif: Mendukung, setuju, atau menunjukkan keinginan untuk "kabur". (misal: "mending pindah", "capek bgt sama konoha", "pajak gila")
    - Negatif: Menentang, skeptis, atau tidak setuju dengan ide "kabur". (misal: "di luar juga susah", "mending bangun bangsa", "hujan emas negeri orang")
    - Netral: Komentar yang hanya bertanya, informatif (tanpa opini), atau tidak memiliki opini yang jelas. (misal: "pindah ke mana ya?", "WNI di jepang banyak")

    PERINGATAN SARKASME:
    Hati-hati dengan sarkasme! Ini sangat penting.
    1. Komentar seperti "makin cinta indonesia ❤️" (di video tentang pajak) kemungkinan besar adalah sarkasme dan harus dilabeli 'Positif' (karena mendukung ide "kabur").
    2. Komentar seperti "emang di singapur gampang 😂" kemungkinan besar adalah sarkasme dan harus dilabeli 'Negatif' (karena menentang ide "kabur").

    Berikut daftar komentar yang perlu diklasifikasikan:
    {numbered_comments}

    Kembalikan hasil dalam format Python list seperti berikut:
    ["Positif", "Negatif", "Netral", ...]

    Jumlah elemen dalam list harus sama dengan jumlah komentar ({len(comments)}).
    Jangan tambahkan teks lain, jangan tulis kode, jangan tulis ```python, jangan ada komentar tambahan.
    """

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )

    raw_output = response.text.strip()

    # Bersihkan tanda ``` jika gemini mengirimkan format yang tidak sesuai
    raw_output = re.sub(r"^```(?:python)?", "", raw_output)
    raw_output = re.sub(r"```$", "", raw_output).strip()

    # Parsing dengan ambil isi dalam tanda kutip
    try:
        labels = re.findall(r'"(.*?)"', raw_output)
        if not labels:  # Jika tidak ada tanda kutip, coba pisah berdasarkan koma
            labels = [w.strip(" []'\"\n") for w in raw_output.split(",") if w.strip()]
        if len(labels) != len(comments):
            print(f"Jumlah label ({len(labels)}) ≠ jumlah komentar ({len(comments)})")
            if len(labels) < len(comments):
                labels += ["Error"] * (len(comments) - len(labels))
            else:
                labels = labels[:len(comments)]
        return labels
    except Exception as e:
        print("Format output tidak valid, hasil mentah:")
        print(raw_output)
        return ["Error"] * len(comments)

In [6]:
comments = df['Teks_Komentar'].head().tolist()
labels = classify_sentiment_batch(comments)

df_preview = pd.DataFrame({
    "Teks_Komentar": comments,
    "sentiment": labels
})

df_preview

,Teks_Komentar,sentiment
0,"Sempat mikir mau pindah ke negara sebelah, nge...",Negatif
1,"Kalo kabur mau kemana ke Singapur ,emang di Si...",Negatif
2,Klo sudah gelap susah terangnya lebih baik bubar,Positif
3,Pengen kabur tapi gak punya uang😭,Positif
4,Kalau mau pindah ke luar negeri ya silahkan sa...,Netral


In [7]:
def process_sentiment_labeling(df, client, batch_size=25, delay=30):
    all_labels = []  # menyimpan semua label hasil model

    # Loop setiap batch komentar
    for i in range(0, len(df), batch_size):
        # Ambil komentar (PERBAIKI NAMA KOLOM DI SINI)
        batch_comments = df['Teks_Komentar'].iloc[i:i+batch_size].astype(str).tolist()
        print(f"\nMemproses batch {i//batch_size + 1} ({len(batch_comments)} komentar)...")

        while True:
            try:
                labels = classify_sentiment_batch(batch_comments)
                break  # keluar dari loop jika berhasil
            except Exception as e:
                if "RESOURCE_EXHAUSTED" in str(e) or "429" in str(e):
                    print("Terkena limit API (429). Menunggu 120 detik sebelum mencoba ulang batch ini...")
                    time.sleep(120)
                else:
                    print(f"Terjadi error pada batch {i//batch_size + 1}: {e}")
                    labels = ["Error"] * len(batch_comments)
                    break  # hentikan loop jika error bukan 429

        # Simpan hasil ke list utama
        all_labels.extend(labels)

        # Jeda antar batch agar tidak limit API request
        if i + batch_size < len(df):
            print(f"Menunggu {delay} detik sebelum lanjut ke batch berikutnya.")
            time.sleep(delay)

    df_result = pd.DataFrame({
        "comment": df["Teks_Komentar"], # PERBAIKI NAMA KOLOM DI SINI JUGA
        "sentiment": all_labels
    })
    
    return df_result

In [8]:
df_result = process_sentiment_labeling(df, client, batch_size=50, delay=60)


Memproses batch 1 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 2 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 3 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 4 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 5 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 6 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 7 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 8 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 9 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 10 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 11 (50 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Mempros

In [10]:
# Baca kedua file
df_cleaned = pd.read_csv("data_filtered.csv")

df_cleaned['sentiment'] = df_result['sentiment']

df_cleaned.to_csv("dataset_labeled.csv", index=False, encoding="utf-8")

print("Dataset final selesai disimpan ke dataset_labeled.csv")

Dataset final selesai disimpan ke dataset_labeled.csv
